# Анализ датасетов, сгенерированных моделью DeepSeek Chat, с помощью стандартных классификаторов

In [90]:
FILE_NAME_PREFIX = 'deepseek_essays'
DF_SPLIT = 9000
DF_COUNT = 5
THRESHOLD=0.99
SEED = 42

In [91]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, precision_recall_curve, confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch
import random
from collections import defaultdict
import torch.nn.functional as F

In [92]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [93]:
df = pd.read_csv("../datasets/human_essays.csv", encoding="utf-8").assign(label=0)[["text", "label"]]

In [94]:
def get_dataset(orig_df: pd.DataFrame, filename: str) -> pd.DataFrame:
    """
    Загружает датасет из CSV, добавляет метку и объединяет с исходным DataFrame. Затем перемешивает строки.

    :param orig_df: исходный DataFrame
    :param filename: путь к CSV-файлу
    :return: объединённый и перемешанный DataFrame
    """
    new_data = pd.read_csv(filename, encoding="utf-8").assign(label=1)[["text", "label"]]

    result = pd.concat([orig_df, new_data], ignore_index=True).sample(frac=1, random_state=SEED).reset_index(drop=True)

    return result

In [95]:
datasets = [
    get_dataset(df[0:DF_SPLIT], f"../datasets/{FILE_NAME_PREFIX}_{i}.csv")
    for i in range(1, DF_COUNT + 1)
]

In [96]:
sizes = [dataset.shape for dataset in datasets]
print(sizes)

[(12000, 2), (12000, 2), (11999, 2), (12000, 2), (12000, 2)]


In [97]:
datasets[0].head(20)

,text,label
0,Table of Contents\n 1. Faith Provides Life’s M...,0
1,Organizational change is often associated with...,0
2,Table of Contents\n 1. Introduction\n 2. EMR I...,0
3,The dramatic festivals of ancient Athens were ...,1
4,Gnomial Functions Inc. wants to determine the ...,0
5,The concept of divine judgement stands as a co...,1
6,"Table of Contents\n 1. The Event, Key Dates, a...",0
7,Table of Contents\n 1. Introduction\n 2. Forma...,0
8,Patients with diabetes mellitus almost always ...,0
9,"Of all types of entertainment, sports games su...",0


### Метрика

In [98]:
def recall_at_precision(y_true, y_scores, min_precision=0.995):
    precision, recall, thresholds = precision_recall_curve(
        y_true, y_scores
    )

    valid = precision[:-1] >= min_precision

    if not np.any(valid):
        return {
            "recall": 0.0,
            "precision": float(precision.max()),
            "threshold": None,
            "threshold_found": False
        }

    idx = np.argmax(recall[:-1][valid])
    valid_indices = np.where(valid)[0]
    best_idx = valid_indices[idx]

    return {
        "recall": float(recall[best_idx]),
        "precision": float(precision[best_idx]),
        "threshold": float(thresholds[best_idx]),
        "threshold_found": True
    }

def fp_metrics(y_true, y_scores, threshold):
    y_pred = (y_scores >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    return {
        "fp": int(fp),
        "fpr": fp / (fp + tn + 1e-9),
        "precision": tp / (tp + fp + 1e-9),
        "recall": tp / (tp + fn + 1e-9),
    }

### LogisticRegression + TF-IDF

In [99]:
def show_top_features(model, vectorizer, top_n=20):
    feature_names = vectorizer.get_feature_names_out()
    coefs = model.coef_[0]
    
    top_positive_idx = coefs.argsort()[::-1][:top_n]
    top_negative_idx = coefs.argsort()[:top_n]

    print("TOP words =======================")
    print("Top positive words:")
    for i in top_positive_idx:
        print(f"{feature_names[i]}: {coefs[i]:.4f}")
    
    print("\nTop negative words:")
    for i in top_negative_idx:
        print(f"{feature_names[i]}: {coefs[i]:.4f}")

In [106]:
def get_log_classification_report(data):
    X_train, X_test, y_train, y_test = train_test_split(
        data["text"],
        data["label"],
        test_size=0.2,
        random_state=SEED,
        stratify=data["label"],
    )

    from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS  
    custom_stop_words = ["web", "2018", "2019", "2020", "2017", "2021", "2016", "al", "et", "et al"]
    all_stop_words = list(ENGLISH_STOP_WORDS.union(custom_stop_words))

    vectorizer = TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        stop_words=all_stop_words
    )

    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    model = LogisticRegression(max_iter=1000, random_state=SEED)
    model.fit(X_train_vec, y_train)

    y_scores = model.predict_proba(X_test_vec)[:, 1]

    rap = recall_at_precision(
        y_test,
        y_scores,
        min_precision=THRESHOLD
    )

    if rap["threshold_found"]:
        y_pred = (y_scores >= rap["threshold"]).astype(int)
        fp = fp_metrics(y_test, y_scores, rap["threshold"])
    else:
        y_pred = np.zeros_like(y_scores, dtype=int)
        fp = None

    show_top_features(model, vectorizer, top_n=15)

    return classification_report(y_test, y_pred), rap, fp

In [107]:
log_results = []
for df in datasets:
    log_results.append(get_log_classification_report(df))

TOP words =======================
Top positive words:
ultimately: 5.6045
profound: 5.3635
merely: 4.5388
like: 4.2667
complex: 3.3850
critical: 3.1356
furthermore: 2.9599
narrative: 2.9329
fundamental: 2.8823
foundational: 2.8596
fundamentally: 2.8095
core: 2.7160
powerful: 2.5399
strategic: 2.3604
underscores: 2.3498

Top negative words:
people: -5.3776
references: -2.8956
used: -2.4735
according: -2.3847
various: -2.3820
journal: -2.2876
main: -2.2474
help: -2.2078
different: -2.1963
information: -2.0806
important: -2.0545
reference: -2.0538
example: -2.0133
author: -1.8983
addition: -1.8967
TOP words =======================
Top positive words:
background context: 5.3877
context: 5.0184
background: 4.3548
like: 4.1996
profound: 3.7389
complex: 3.4040
critical: 3.1563
narrative: 3.0616
merely: 3.0478
core: 2.9605
conclusion: 2.9340
foundational: 2.8748
strategic: 2.8354
systemic: 2.5937
particularly: 2.5429

Top negative words:
people: -5.6318
references: -2.6742
journal: -2.4998
acco

In [108]:
rows = []
for i, (report, rap, fp) in enumerate(log_results, start=1):
    print(f"Отчёт по датасету №{i}\n")
    print(report)

    rows.append({
        "dataset": i,
        "recall@precision": rap["recall"],
        "threshold_found": rap["threshold_found"],
        "fp": fp["fp"],
        "fpr": fp["fpr"],
        "precision": fp["precision"],
        "recall": fp["recall"],
    })

df_metrics = pd.DataFrame(rows)
print("Сводная таблица\n")
print(df_metrics.round(4).to_string(index=False))

Отчёт по датасету №1

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      1.00      1.00       600

    accuracy                           1.00      2400
   macro avg       1.00      1.00      1.00      2400
weighted avg       1.00      1.00      1.00      2400

Отчёт по датасету №2

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      1.00      1.00       600

    accuracy                           1.00      2400
   macro avg       1.00      1.00      1.00      2400
weighted avg       1.00      1.00      1.00      2400

Отчёт по датасету №3

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      1.00      0.99       600

    accuracy                           1.00      2400
   macro avg       0.99      1.00      1.00      2400
weighted

### Boosting

In [13]:
def get_boost_classification_report(data):
    df = data.copy()
    df["tokens"] = df["text"].apply(lambda text: simple_preprocess(str(text)))

    w2v_model = Word2Vec(
        sentences=df["tokens"],
        vector_size=300,
        window=7,
        min_count=2,
        workers=12,
        sg=1,
    )

    def text_to_vector(tokens, model):
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

    X = np.vstack([text_to_vector(tokens, w2v_model) for tokens in df["tokens"]])
    y = df["label"].values

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=SEED,
        stratify=y,
    )

    model = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.5,
        eval_metric="logloss",
        n_jobs=12,
    )

    model.fit(X_train, y_train)

    y_scores = model.predict_proba(X_test)[:, 1]

    rap = recall_at_precision(y_test, y_scores, min_precision=THRESHOLD)

    if rap["threshold_found"]:
        y_pred = (y_scores >= rap["threshold"]).astype(int)
        fp = fp_metrics(y_test, y_scores, rap["threshold"])
    else:
        y_pred = np.zeros_like(y_scores, dtype=int)
        fp = None

    return classification_report(y_test, y_pred), rap, fp

In [14]:
boost_results = []
for df in datasets:
    boost_results.append(get_boost_classification_report(df))

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_fl

In [15]:
rows = []
for i, (report, rap, fp) in enumerate(boost_results, start=1):
    print(f"Отчёт по датасету №{i}\n")
    print(report)

    rows.append({
        "dataset": i,
        "recall@precision": rap["recall"],
        "threshold_found": rap["threshold_found"],
        "fp": fp["fp"],
        "fpr": fp["fpr"],
        "precision": fp["precision"],
        "recall": fp["recall"],
    })

df_metrics = pd.DataFrame(rows)
print("Сводная таблица\n")
print(df_metrics.round(4).to_string(index=False))

Отчёт по датасету №1

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      0.98      0.99       400

    accuracy                           1.00      2200
   macro avg       0.99      0.99      0.99      2200
weighted avg       1.00      1.00      1.00      2200

Отчёт по датасету №2

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      1.00      0.99       400

    accuracy                           1.00      2200
   macro avg       0.99      1.00      1.00      2200
weighted avg       1.00      1.00      1.00      2200

Отчёт по датасету №3

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      0.98      0.99       400

    accuracy                           1.00      2200
   macro avg       0.99      0.99      0.99      2200
weighted

### BERT

In [16]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [17]:
class ChunkedTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.tolist() if hasattr(labels, "tolist") else list(labels)

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        essay_id = self.encodings["essay_ids"][idx]
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels[essay_id], dtype=torch.long),
            "essay_id": essay_id,
        }

In [18]:
def tokenize_with_sliding_window(texts, tokenizer, max_length=256, stride=128):
    input_ids, attention_masks, essay_ids = [], [], []

    for essay_id, text in enumerate(texts):
        encoded = tokenizer(
            text,
            truncation=True,
            max_length=max_length,
            stride=stride,
            return_overflowing_tokens=True,
            padding="max_length",
            return_tensors="pt",
        )

        n_chunks = encoded["input_ids"].shape[0]
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])
        essay_ids.extend([essay_id] * n_chunks)

    return {
        "input_ids": torch.cat(input_ids),
        "attention_mask": torch.cat(attention_masks),
        "essay_ids": essay_ids,
    }

def get_bert_classification_report(
    data,
    model_name="bert-base-uncased",
    max_length=256,
    stride=128,
    batch_size=32,
    epochs=2,
):
    X_train, X_test, y_train, y_test = train_test_split(
        data["text"],
        data["label"],
        test_size=0.2,
        random_state=SEED,
        stratify=data["label"],
    )

    tokenizer = BertTokenizer.from_pretrained(model_name)

    train_enc = tokenize_with_sliding_window(X_train, tokenizer, max_length, stride)
    test_enc = tokenize_with_sliding_window(X_test, tokenizer, max_length, stride)

    train_dataset = ChunkedTextDataset(train_enc, y_train)
    test_dataset = ChunkedTextDataset(test_enc, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
    ).to(device)

    optimizer = AdamW(model.parameters(), lr=2e-5)

    model.train()
    for epoch in range(epochs):
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items() if k != "essay_id"}
            optimizer.zero_grad()
            outputs = model(**batch)
            outputs.loss.backward()
            optimizer.step()

    model.eval()
    essay_probs = defaultdict(list)
    essay_labels = {}

    with torch.no_grad():
        for batch in test_loader:
            essay_ids = batch["essay_id"]
            labels = batch["labels"]

            batch = {k: v.to(device) for k, v in batch.items() if k != "essay_id"}

            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            probs = F.softmax(outputs.logits, dim=1)[:, 1]

            for i, essay_id in enumerate(essay_ids):
                essay_probs[essay_id].append(probs[i].cpu())
                essay_labels[essay_id] = labels[i].item()

    final_scores, final_labels = [], []

    for essay_id, probs_list in essay_probs.items():
        mean_prob = torch.stack(probs_list).mean().item()
        final_scores.append(mean_prob)
        final_labels.append(essay_labels[essay_id])

    final_scores = np.array(final_scores)
    final_labels = np.array(final_labels)

    rap = recall_at_precision(final_labels, final_scores, min_precision=THRESHOLD)

    if rap["threshold_found"]:
        y_pred = (final_scores >= rap["threshold"]).astype(int)
        fp = fp_metrics(final_labels, final_scores, rap["threshold"])
    else:
        y_pred = np.zeros_like(final_scores, dtype=int)
        fp = None

    return classification_report(final_labels, y_pred), rap, fp

In [19]:
bert_results = []
for df in datasets:
    bert_results.append(get_bert_classification_report(df))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are

In [20]:
rows = []
for i, (report, rap, fp) in enumerate(bert_results, start=1):
    print(f"Отчёт по датасету №{i}\n")
    print(report)

    rows.append({
        "dataset": i,
        "recall@precision": rap["recall"],
        "threshold_found": rap["threshold_found"],
        "fp": fp["fp"],
        "fpr": fp["fpr"],
        "precision": fp["precision"],
        "recall": fp["recall"],
    })

df_metrics = pd.DataFrame(rows)
print("Сводная таблица\n")
print(df_metrics.round(4).to_string(index=False))

Отчёт по датасету №1

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      1.00      0.99       400

    accuracy                           1.00      2200
   macro avg       0.99      1.00      1.00      2200
weighted avg       1.00      1.00      1.00      2200

Отчёт по датасету №2

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      1.00      1.00       400

    accuracy                           1.00      2200
   macro avg       1.00      1.00      1.00      2200
weighted avg       1.00      1.00      1.00      2200

Отчёт по датасету №3

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1800
           1       0.99      1.00      1.00       400

    accuracy                           1.00      2200
   macro avg       1.00      1.00      1.00      2200
weighted